# Active Qubit Reset 

In this notebook, we demonstrate how to execute active qubit reset, i.e. active feedback based on real-time measurement of the qubit state. 
We require either a SHFQC instrument for this notebook or a combination of SHFSG and SHFQA connected via a PQSC. 

This demonstration runs without real qubits, assuming a loopback on the readout drive line directly into the reaoud acquisition line. We emulate the different qubit states by two different readout measurement pulses, differing by a phase. 
To demonstrate real-time feedback, we first calibrate the state discrimintation unit for the two measurement pulsese we choose to emulate the qubit response. The we use this calibration to play an arbitrary simulated pattern of qubit states and demonstrate the real-time feedback capabilities of the instrument.  

## 0. General Imports and Definitions

### 0.1 Python Imports 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# all LabOne Q functionality
from laboneq.simple import *

# helper import
from laboneq.contrib.example_helpers.feedback_helper import (
    complex_freq_phase,
    exp_raw,
    exp_integration,
    exp_discrimination,
)
from laboneq.contrib.example_helpers.descriptors.shfqc import descriptor_shfqc
from laboneq.contrib.example_helpers.descriptors.shfsg_shfqa_pqsc import (
    descriptor_shfsg_shfqa_pqsc,
)

In [ ]:
use_emulation = True

In [ ]:
compiler_settings = {
    "SHFSG_FORCE_COMMAND_TABLE": True,
    "SHFSG_MIN_PLAYWAVE_HINT": 32,
    "SHFSG_MIN_PLAYZERO_HINT": 32,
}

## 1. Define the Device Setup and apply baseline calibration

We'll load a descriptor file to define our device setup and logical signal lines and then apply a baseline calibration to the signal lines based on a dictionary of qubit parameters

### 1.1 DeviceSetup from descriptor

In [ ]:
# Define and Load a Device Setup

# Choose your setup - local feedback on a standalone SHFQC or ...
# feedback_type="local"
# my_descriptor = descriptor_shfqc
# ... global feedback on a combination of SHFSG and SHFQA, connected through a PQSC
feedback_type = "global"
my_descriptor = descriptor_shfsg_shfqa_pqsc


my_setup = DeviceSetup.from_descriptor(
    my_descriptor,
    server_host="my_ip_address",  # ip address of the LabOne dataserver used to communicate with the instruments
    server_port="8004",  # port number of the dataserver - default is 8004
    setup_name="QC_standalone",  # setup name
)

### 1.2 Baseline calibration parameters as dictionary

In [ ]:
base_qubit_parameters = {
    "resonance_frequency": 1100e6,  # qubit drive frequency in [Hz] - relative to local oscillator for qubit drive upconversion
    "readout_resonator_frequency": 900e6,
    "readout_length": 400e-9,
    "readout_amplitude": 0.4,
    "readout_integration_delay": 20e-9,
    "pi_amplitude": 0.3,
    "pi_2_amplitude": 0.1,
    "pulse_length": 200e-9,
    "readout_data_delay": 100e-9,
    # local oscillator settings
    "readout_lo_frequency": 1.0e9,  # readout LO Frequency
    "readout_range_out": 5,
    "readout_range_in": 10,
    "drive_lo_frequency": 1.0e9,  # drive LO frequencies, one center frequency per two channels
    "drive_range": 10,
}

In [ ]:
# define qubit object, containing all relevant information for the tuneup experiments
my_parameters = QubitParameters(
    resonance_frequency=base_qubit_parameters["resonance_frequency"],
    drive_lo_frequency=base_qubit_parameters["drive_lo_frequency"],
    readout_resonator_frequency=base_qubit_parameters["readout_resonator_frequency"],
    readout_lo_frequency=base_qubit_parameters["readout_lo_frequency"],
    readout_integration_delay=base_qubit_parameters["readout_integration_delay"],
    drive_range=base_qubit_parameters["drive_range"],
    readout_range_out=base_qubit_parameters["readout_range_out"],
    readout_range_in=base_qubit_parameters["readout_range_in"],
    user_defined={
        "readout_length": base_qubit_parameters["readout_length"],
        "readout_amplitude": base_qubit_parameters["readout_amplitude"],
        "pi_amplitude": base_qubit_parameters["pi_amplitude"],
        "pi_2_amplitude": base_qubit_parameters["pi_2_amplitude"],
        "pulse_length": base_qubit_parameters["pulse_length"],
        "readout_data_delay": base_qubit_parameters["readout_data_delay"],
    },
)
# my_qubit = Qubit(0, base_qubit_parameters)
my_qubit = Qubit.from_logical_signal_group(
    "q0", my_setup.logical_signal_groups["q0"], my_parameters
)

### 1.3 Calibration with Dummy Emulation state

Below, we define a calibration using an extra measure line to emulate the excited state of the qubit. In a real experiment, the calibration can be generated from your qubit object with `my_setup.set_calibration(my_qubit.calibration())`, without the need to sepcify and set the calibration as we do here.

In [ ]:
# generate baseline device calibration
my_base_calibration = Calibration()
# qubit drive line
my_base_calibration[
    f"/logical_signal_groups/{my_qubit.uid}/drive_line"
] = SignalCalibration(
    oscillator=Oscillator(
        frequency=my_qubit.parameters.drive_frequency,
        modulation_type=ModulationType.HARDWARE,
    ),
    local_oscillator=Oscillator(
        frequency=my_qubit.parameters.drive_lo_frequency,
    ),
    range=my_qubit.parameters.drive_range,
)
# qubit measure line - for pulse emulating state 0
my_base_calibration[
    f"/logical_signal_groups/{my_qubit.uid}/measure_line"
] = SignalCalibration(
    oscillator=Oscillator(
        frequency=my_qubit.parameters.readout_frequency,
        modulation_type=ModulationType.SOFTWARE,
    ),
    local_oscillator=Oscillator(
        frequency=my_qubit.parameters.readout_lo_frequency,
    ),
    range=my_qubit.parameters.readout_range_out,
)
# qubit measure line - for pulse emulating state 1
my_base_calibration[f"/logical_signal_groups/q1/measure_line"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=my_qubit.parameters.readout_frequency,
        modulation_type=ModulationType.SOFTWARE,
    ),
    local_oscillator=Oscillator(
        frequency=my_qubit.parameters.readout_lo_frequency,
    ),
    range=my_qubit.parameters.readout_range_out,
)
# qubit acquire line - no baseband modulation applied
my_base_calibration[
    f"/logical_signal_groups/{my_qubit.uid}/acquire_line"
] = SignalCalibration(
    oscillator=None,
    local_oscillator=Oscillator(
        frequency=my_qubit.parameters.readout_lo_frequency,
    ),
    range=my_qubit.parameters.readout_range_in,
    port_delay=my_qubit.parameters.readout_integration_delay,
)

In [ ]:
# apply dummy calibration to device setup
# comment out if not emulating
my_setup.set_calibration(my_base_calibration)

# apply calibration from qubit object
# my_setup.set_calibration(my_qubit.calibration())

q0 = my_setup.logical_signal_groups["q0"].logical_signals
q1 = my_setup.logical_signal_groups["q1"].logical_signals

In [ ]:
# create and connect to a LabOne Q session
my_session = Session(device_setup=my_setup)
my_session.connect(do_emulation=use_emulation)

## 2. Calibration of state discrimination

We determine the optimal integration weights by subtracting and conjugating the raw response corresponding to the two different qubit states. We then additionall rotate these integration weights to result in maximum separation of the resulting IQ valuebs on the real axis and set the threshold to the setup calibration.

### 2.1 Define measurement pulse waveforms to simulate measurement of |0> and |1> qubit states

In [ ]:
# measure pulse parameters
pulse_len = my_qubit.parameters.user_defined["readout_length"]
pulse_phase = np.pi / 4

# sampling rate of SHFQC
sampling_rate = 2.0e9

pulse_freq = 0.0
measure0_gen2 = pulse_library.sampled_pulse_complex(
    complex_freq_phase(
        sampling_rate,
        pulse_len,
        pulse_freq,
        my_qubit.parameters.user_defined["readout_amplitude"],
        0,
    )
)
measure1_gen2 = pulse_library.sampled_pulse_complex(
    complex_freq_phase(
        sampling_rate,
        pulse_len,
        pulse_freq,
        my_qubit.parameters.user_defined["readout_amplitude"],
        pulse_phase,
    )
)

### 2.2 Determine optimal integration weights based on raw readout results of two measurement pulses

In [ ]:
## Raw |0>
r = my_session.run(exp_raw(measure_pulse=measure0_gen2, q0=q0, pulse_len=pulse_len))
raw0 = r.acquired_results["raw"].data

## Raw |1>
r = my_session.run(exp_raw(measure_pulse=measure1_gen2, q0=q0, pulse_len=pulse_len))
raw1 = r.acquired_results["raw"].data

## optimal integration kernel
samples_kernel = np.conj(raw1 - raw0)
# plt.figure()
# plt.plot(samples_kernel.real, samples_kernel.imag)
plt.figure()
plt.plot(samples_kernel.real)
plt.plot(samples_kernel.imag)

### 2.3 Determine optimal rotation of integration weights and discrimination threshold

In [ ]:
do_rotation = True

my_exp = exp_integration(
    measure0=measure0_gen2,
    measure1=measure1_gen2,
    q0=q0,
    q1=q1,
    samples_kernel=samples_kernel,
)

r = my_session.run(my_exp)
res0 = r.acquired_results["data0"].data
res1 = r.acquired_results["data1"].data

connect_vector = np.median(res1) - np.median(res0)
if do_rotation:
    rotation_angle = -np.angle(connect_vector)
else:
    rotation_angle = 0

res0_rot = res0 * np.exp(1j * rotation_angle)
res1_rot = res1 * np.exp(1j * rotation_angle)

my_threshold = (np.median(res0_rot.real) + np.median(res1_rot.real)) / 2

if do_rotation:
    plt.scatter(res0.real, res0.imag, c="k", alpha=0.1)
    plt.scatter(res1.real, res1.imag, c="g", alpha=0.1)

plt.scatter(res0_rot.real, res0_rot.imag, c="b")
plt.scatter(res1_rot.real, res1_rot.imag, c="r")
plt.plot(
    [my_threshold, my_threshold],
    [
        min([*res0_rot.imag, *res1_rot.imag, *res0.imag, *res1.imag]),
        max([*res0_rot.imag, *res1_rot.imag, *res0.imag, *res1.imag]),
    ],
    "r",
)
if do_rotation:
    print(f"Using threshold = {my_threshold:e} and rotation angle: {rotation_angle:e}")
else:
    print(f"Using threshold={my_threshold:e}")

In [ ]:
## define properly rotated integration kernel and set state discrimination threshold in device setup calibration
my_integration_weights = pulse_library.sampled_pulse_complex(
    samples_kernel * np.exp(1j * rotation_angle)
)

q0["acquire_line"].calibration.threshold = my_threshold

### 2.4 Check status of state discrimination calibration

#### 2.4.1 Check for proper rotation of kernel

IQ values should be maximally separate on the real axis

In [ ]:
my_other_exp = exp_integration(
    measure0=measure0_gen2,
    measure1=measure1_gen2,
    q0=q0,
    q1=q1,
    samples_kernel=samples_kernel,
    rotation_angle=rotation_angle,
)

r = my_session.run(my_other_exp)

res0 = r.acquired_results["data0"].data
res1 = r.acquired_results["data1"].data

connect_vector = np.median(res1) - np.median(res0)

threshold_rot = (np.median(res0.real) + np.median(res1.real)) / 2

plt.scatter(res0.real, res0.imag, c="b")
plt.scatter(res1.real, res1.imag, c="r")

plt.plot(
    [threshold_rot, threshold_rot],
    [min([*res0.imag, *res1.imag]), max([*res0.imag, *res1.imag])],
    "r",
)

print(f"Using threshold={threshold_rot:e}")

#### 2.4.2 Check correct state discrimination when including rotation of integration weights

In [ ]:
r = my_session.run(
    exp_discrimination(
        measure0=measure0_gen2,
        measure1=measure1_gen2,
        q0=q0,
        q1=q1,
        samples_kernel=samples_kernel,
        threshold=my_threshold,
        rotation_angle=rotation_angle,
    )
)
s0 = r.acquired_results["data0"].data
s1 = r.acquired_results["data1"].data

plt.plot(s0.real, ".b")
plt.plot(s1.real, ".r")

## 3. Feedback experiment

Here we create a real-time feedback demonstration that plays back a user defined sequence of "qubit states" i.e. a sequences of different measurement pulses emulating different qubit states. The measured qubit state after state discrimination is used in a real-time feedback section to playback either of two pulses: x90 for the qubit in its ground state and x180 for the qubit in the excited state. 

### 3.0 Define Pulses

In [ ]:
x90 = pulse_library.drag(
    uid=f"x90_q{id}",
    length=my_qubit.parameters.user_defined["pulse_length"],
    amplitude=my_qubit.parameters.user_defined["pi_2_amplitude"],
    sigma=0.3,
    beta=0.4,
)
x180 = pulse_library.drag(
    uid=f"x180_q{id}",
    length=my_qubit.parameters.user_defined["pulse_length"],
    amplitude=my_qubit.parameters.user_defined["pi_amplitude"],
    sigma=0.3,
    beta=0.4,
)

### 3.1 Define Experiment

In [ ]:
def create_feedback_experiment(
    feedback_pattern="1010111",
    num_average=2,
    ## delay parameter between state readout and reset playback, needs to be minimal 120ns for local feedback and 400ns for global feedback
    acquire_delay=120e-9,
    # parameters to simulate the qubit state discrimination
    measure_pulse0=measure0_gen2,
    measure_pulse1=measure1_gen2,
    integration_weights=my_integration_weights,
    acquisition_type=AcquisitionType.DISCRIMINATION,
    # parameters that determine the type of pulse sequence to be played
    x90=x90,
    x180=x180,
    pattern_delay=1e-6,
):
    exp = Experiment(
        signals=[
            ExperimentSignal("drive"),
            ExperimentSignal("measure0"),
            ExperimentSignal("measure1"),
            ExperimentSignal("acquire"),
        ]
    )
    # ensure delay time is set large enough to allow for signal processing and data delivery
    if feedback_type == "local" and acquire_delay < 120e-9:
        print(
            "Local feedback requires a longer additional delay time than specified, setting to 120ns."
        )
        acquire_delay = 120e-9
    elif feedback_type == "global" and acquire_delay < 400e-9:
        print(
            "Global feedback requires a longer additional delay time than specified, setting to 400ns."
        )
        acquire_delay = 400e-9

    with exp.acquire_loop_rt(
        count=num_average,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=acquisition_type,
    ):
        # iterate over the letters of the given pattern
        for id, letter in enumerate(feedback_pattern):
            # placeholder for experiments / pulse sequences on the qubit
            with exp.section(uid=f"drive_{id}"):
                exp.delay(signal="drive", time=5 * x90.length)
            # qubit state readout
            with exp.section(uid=f"measure_{id}", play_after=f"drive_{id}"):
                # emulate qubit state by playing different measurement pulses based on pattern
                if letter == "0":
                    exp.play(signal="measure0", pulse=measure_pulse0)
                else:
                    exp.play(signal="measure1", pulse=measure_pulse1)
                # acquire result, assign to handle
                exp.acquire(
                    signal="acquire",
                    handle="qubit_state",
                    kernel=integration_weights,
                )
                # delay after state discrimination and before reset pulse playback
                exp.delay(signal="acquire", time=acquire_delay)
            # real-time feedback, fetching the measurement data identified by handle from the QA unit specified in the descriptor
            # determines automatically if local (SHFQC only) of global (through PQSC) feedback path is to be used
            with exp.match(
                uid=f"feedback_{id}",
                handle="qubit_state",
                play_after=f"measure_{id}",
            ):
                # measurement result 0 - ground state
                with exp.case(state=0):
                    # could be "pass". i.e. doing nothing. Here we instead play a x90 pulse, purely for visual reasons
                    exp.play(signal="drive", pulse=x90)
                # measurement result 0 - excited state
                with exp.case(state=1):
                    # play x180 pulse
                    exp.play(signal="drive", pulse=x180)
        # introduce a delay between repetitions of the pattern, only for visual distinction
        with exp.section():
            exp.delay(signal="drive", time=pattern_delay)

    return exp

In [ ]:
my_signal_map = {
    "drive": q0["drive_line"],
    "measure0": q0["measure_line"],
    "measure1": q1["measure_line"],
    "acquire": q0["acquire_line"],
}

### 3.3 Run experiment

In [ ]:
my_feedback_exp = create_feedback_experiment(
    feedback_pattern="1010111",
    acquire_delay=150e-9,
    acquisition_type=AcquisitionType.INTEGRATION,
)
my_feedback_exp.set_signal_map(my_signal_map)

In [ ]:
# compile experiment
my_compiled_exp = my_session.compile(my_feedback_exp)

In [ ]:
# run experiment and get the results
my_results = my_session.run(my_compiled_exp)

In [ ]:
# when executed in integration mode, IQ data of each state readout is still available
my_data = my_results.get_data("qubit_state")
my_data

In [ ]:
## Look at th pulse sheet - feedback is characterised by two simultaneous sections
# show_pulse_sheet("feedback_experiment", my_compiled_exp)

In [ ]:
## have a look at the sequencer code for the QA unit, making the measurements
print(my_compiled_exp.src[0]["text"])

In [ ]:
## have a look at the sequencer code for the SG unit, playing the feedback pulses
print(my_compiled_exp.src[1]["text"])